In [2]:
#Import the Required Libraries
import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.datasets import fetch_mldata
from keras.datasets import mnist
import numpy as np

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data

training_data = mnist.load_data()

# Batch Learning Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 50
num_train = training_data.train.num_examples
num_batches = (num_train // batch_size) + 1
epochs = 2

# RNN LSTM Network Parameters
n_input = 28  # MNIST data input (img shape: 28*28)
n_steps = 28  # timesteps
n_hidden = 128  # hidden layer num of features
n_classes = 10  # MNIST total classes (0-9 digits)


Using TensorFlow backend.


    8192/11490434 [..............................] - ETA: 12:19

   24576/11490434 [..............................] - ETA: 6:09 

   40960/11490434 [..............................] - ETA: 4:54

   73728/11490434 [..............................] - ETA: 3:23

...

11116544/11490434 [============================>.] - ETA: 0s

11460608/11490434 [============================>.] - ETA: 0s

11476992/11490434 [============================>.] - ETA: 0s

11493376/11490434 [==============================] - 11s 1us/step


AttributeError: 'tuple' object has no attribute 'train'

In [ ]:
# Define th forward pass for the RNN
def RNN(x, weights, biases):
    
    # Unstack to get a list of n_stepstensors of shape (batch_size, n_input) as illustrated in Figure 4-12
    x = tf.unstack(x, n_steps, 1)
    
    # Define a lstm cell
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [ ]:
# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0

    while i < epochs:
        for step in range(num_batches):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
            if (step + 1) % display_step == 0:
                # Calculate batch accuracy
                acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
                # Calculate batch loss
                loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
                print "Epoch: " + str(i+1) + ",step:"+ str(step+1) +", Minibatch Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc)
        i += 1
    print "Optimization Finished!"
    
    # Calculate accuracy
    test_len = 500
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label})